# 前置

In [6]:
import os
import pandas as pd

# 設定工作資料夾
work_folder = '【正式】202405'

# 設定搜尋新聞區間是過去一周還是一個月
m_or_w = 'm'

# 計時
import time
start_time = time.time()

In [7]:
# 設定搜尋關鍵字
keyword = ["IFRS17", "ICS2.0", "利率決策", "FED", "台灣CPI", "金管會", "央行",
"美元", "外匯", "美國PCE", "美國非農就業", "美國十年期公債殖利率"]
'''
["台北富邦銀行", "玉山銀行", "中信銀行", "兆豐銀行", "國泰世華", "台新銀行"]

"渣打銀行", "凱基銀行", "新光銀行", "華南銀行",
"第一銀行", "永豐銀行", "元大銀行", "土地銀行", "合庫", "土地銀行",
"臺銀", "花旗台灣", "匯豐台灣", "星展銀行", "樂天國際銀行", "連線銀行", "將來銀行"
'''

'''
["IFRS17", "ICS2.0", "利率決策", "FED", "台灣CPI", "金管會", "央行",
"美元", "外匯", "美國PCE", "美國非農就業", "美國十年期公債殖利率"]
'''

'\n["IFRS17", "ICS2.0", "利率決策", "FED", "台灣CPI", "金管會", "央行",\n"美元", "外匯", "美國PCE", "美國非農就業", "美國十年期公債殖利率"]\n'

In [8]:
# 爬蟲
import requests
from bs4 import BeautifulSoup
# 爬蟲禮儀
import random
# 時間轉譯
import dateparser
# 假headers
from fake_useragent import UserAgent

In [9]:
!pip install dateparser
!pip install fake_useragent

# 文字轉換成Token
!pip install -q -U tiktoken
# ChatGPT套件
!pip install -q -U openai
# chatgpt模組
from openai import OpenAI

# 進度條所需套件
import sys

# token轉換模組（配合API請求上限）
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

# 其他模組
import ast
import re

In [10]:
date_info = {'奇摩股市':["div", "caas-attr-time-style"],#
        '聯合新聞網':["section", "article-content__editor", "p"],
        '經濟日報':["time", "article-body__time"],
        'Anue鉅亨':["main", "c1tt5pk2", "section"],
        '奇摩新聞':["div", "caas-attr-time-style"],#
        '自由財經':["div", "whitecon boxTitle boxText", "p"],
        'Yahoo新聞':["div", "caas-attr-time-style"]}#

'''檢查url的soup'''
# 設定搜尋連結
url = f"https://money.udn.com/money/story/5613/7771359"
print(url)

# 建立隨機 headers
ua = UserAgent()
headers = {'User-Agent':ua.random}

# 取得網頁內容
r = requests.get(url, headers=headers)

# 解析網頁
soup = BeautifulSoup(r.text, 'html.parser')
# print(soup)
aas = soup.find('time', class_ = "article-body__time")
aas.text
# for i in soup.find_all('span', class_='oWL0vf'): 
#     number = re.findall('\d+', i.text)[0]
#     print(number)

https://money.udn.com/money/story/5613/7771359


'2024/02/15 20:32:42'

# 000_爬取新聞URL

In [11]:
output_folder = '000_新聞URL'
start_time = time.time()

In [12]:
# 檢查是否已無搜尋結果
def check_no_result_(soup):
    maybe_nan = soup.find_all('div', class_="card-section")
    for c in maybe_nan:
        if '找不到符合的' in str(c):
            # 回傳搜尋盡頭的頁數
            for i in soup.find_all('span', class_='oWL0vf'): 
              a = re.findall('\d+', i.text)[0]
              return True, a
    return False, '30'
        
def get_urls(keyword, max_N):
    title = []
    date = []
    urls = []
    ref = []
    # google搜尋 指定條件分別為關鍵字、第幾筆、期間開始日、期間結束日、時間倒序sbd、新聞搜尋tbm
    for num in range(0, max_N, 8):
      # 設定搜尋連結
      url = f"https://www.google.com/search?q={keyword}&start={num}&tbs=sbd:1,qdr:{m_or_w},sbd:1&tbm=nws"
      print(url)

      # 建立隨機 headers
      ua = UserAgent()
      headers = {'User-Agent':ua.random}

      # 取得網頁內容
      r = requests.get(url, headers=headers)

      # 爬蟲禮儀
      time.sleep(random.uniform(5, 10))

      # 解析網頁
      soup = BeautifulSoup(r.text, 'html.parser')
      # print(soup)
      check, page_number = check_no_result_(soup)
      print(check, page_number)
      # 先檢查有無搜尋結果 # 無搜尋結果便跳出
      if  check:
          break
      else:
          # 尋找新聞標題
          for i in soup.find_all('div', class_=['n0jPhd ynAwRc MBeuO nDgy9d', 'n0jPhd ynAwRc tNxQIb nDgy9d']): # 觀察原始碼得知，新聞標題在div tag裡面，class為n0jPhd ynAwRc MBeuO nDgy9d
              title.append(i.text)
          # 尋找日期
          for i in soup.find_all('div', class_="OSrXXb rbYSKb LfVVr"):
              i = dateparser.parse(i.text)
              date.append(i)
              print(i)
          # 尋找連結 # 連結藏在tag裡面而非text裡面，所以要額外get
          for i in soup.find_all('a', class_="WlydOe"):
              urls.append(i.get('href'))
          # 尋找出處
          for i in soup.find_all('div', class_="MgUUmf NUnG9d"):
              ref.append(i.text)
    return   title, date, urls, ref, page_number

# 迴圈搜尋
def get_keyword_url(keyword):
  df = pd.DataFrame()
  page_number ='30'
  max_N = 1024
  # google搜尋結果最多30頁，當檢查已無搜尋結果卻
  while page_number=='30':
    title, date, urls, ref, page_number = get_urls(keyword, max_N)

    # 建立pd格式
    temp_df = pd.DataFrame({
        'title': title,
        'date': date,
        'urls': urls,
        'ref': ref
    })
    df = pd.concat([df, temp_df], axis=0)

  # 去重（標題重複者刪除，保留第一列，直接取代原檔案）
  df.drop_duplicates(subset=['title'], keep='first', inplace=True)
  # 重新排序index，因去重的index會直接拿掉，後面的不會遞補，因此需重排
  df.reset_index(inplace=True, drop=True)

  # df轉存為csv檔，因編碼問題須特別使用UTF-8-Sig
  df.to_csv(rf"C:\Users\RYAN\OneDrive\桌面\Python\Capstone_玉山銀行v3\{work_folder}\000_新聞URL\{keyword}_url.csv", encoding = 'UTF-8-Sig', index=False)

  return df

# 建立待處理清單
def creat_list_to_run_(input_folder: str, output_folder: str):
  # 獲取各關鍵字之檔名
  
  file_list = os.listdir(rf'C:\Users\RYAN\OneDrive\桌面\Python\Capstone_玉山銀行v3\{work_folder}\{input_folder}')

  # 檢查是否已run過
  to_run_list = []
  ending_list = os.listdir(rf'C:\Users\RYAN\OneDrive\桌面\Python\Capstone_玉山銀行v3\{work_folder}\{output_folder}')
  for i in file_list:
    if i.split('_')[0] not in list(map(lambda x: x.split('_')[0], ending_list)):
      to_run_list.append(i)
  return to_run_list

In [71]:
# 按關鍵字逐一且逐期搜尋
for i in keyword:
  df = get_keyword_url(i)
  print(f"{i}筆數為 {len(df['title'])} 筆")

https://www.google.com/search?q=IFRS17&start=0&tbs=sbd:1,qdr:m,sbd:1&tbm=nws
False 30
2024-05-23 10:55:32.608928
2024-05-16 10:55:32.613932
https://www.google.com/search?q=IFRS17&start=8&tbs=sbd:1,qdr:m,sbd:1&tbm=nws
True 2
IFRS17筆數為 2 筆
https://www.google.com/search?q=ICS2.0&start=0&tbs=sbd:1,qdr:m,sbd:1&tbm=nws
False 30
2024-05-23 10:55:49.485904
2024-05-23 10:55:49.488906
2024-05-23 10:55:49.490906
2024-05-23 10:55:49.493916
2024-05-16 10:55:49.496909
https://www.google.com/search?q=ICS2.0&start=8&tbs=sbd:1,qdr:m,sbd:1&tbm=nws
False 30
https://www.google.com/search?q=ICS2.0&start=16&tbs=sbd:1,qdr:m,sbd:1&tbm=nws
True 2
ICS2.0筆數為 5 筆
https://www.google.com/search?q=利率決策&start=0&tbs=sbd:1,qdr:m,sbd:1&tbm=nws
False 30
2024-06-05 10:56:13.179436
2024-06-05 10:56:13.184486
2024-06-05 10:56:13.187429
2024-06-04 10:56:13.189427
2024-06-04 10:56:13.192426
2024-06-04 10:56:13.195962
2024-06-04 10:56:13.198958
2024-06-03 10:56:13.202951
2024-06-03 10:56:13.205999
2024-06-03 10:56:13.210018
ht

In [ ]:
end_time = time.time()
total_time000 = end_time - start_time

# 將 total_time 轉換為秒數
total_seconds = int(total_time000)

# 計算小時數
hours = total_seconds // 3600

# 計算剩餘秒數
remaining_seconds = total_seconds % 3600

# 計算分鐘數
minutes = remaining_seconds // 60

# 計算剩餘秒數
seconds = remaining_seconds % 60

# 格式化輸出
output = f"{hours:02d}hrs{minutes:02d}mins{seconds:02d}secs"
print("程式執行時間：", output)

# 001_爬取內文

In [13]:
input_folder = '000_新聞URL'
output_folder = '001_新聞內文'
start_time = time.time()

In [14]:
# 設定各家媒體之原始碼原文節點
info = {'奇摩股市':["div", "caas-body", "p"],
        '聯合新聞網':["section", "article-content__editor", "p"],
        '經濟日報':["section", "article-body__editor", "p"],
        'Anue鉅亨':["main", "c1tt5pk2", "section"],
        '奇摩新聞':["div", "caas-body", "p"],
        '自由財經':["div", "whitecon boxTitle boxText", "p"],
        'Yahoo新聞':["div", "caas-body", "p"]}

def grabing(info_url, ref, info):

  # 建立隨機 headers
  ua = UserAgent()
  headers = {'User-Agent':ua.random}

  # 取得網頁內容
  url = info_url
  r = requests.get(url, headers=headers)

  # 爬蟲禮儀
  time.sleep(random.uniform(0, 3))

  # 解析網頁
  soup = BeautifulSoup(r.text, 'html.parser')
  # print(soup)

  # 將節點帶出
  info_block_element = info[ref][0]
  info_class = info[ref][1]
  info_paragraph = info[ref][2]

  # 自由財經特殊處理
  recursive = False
  if ref == '自由財經':
    recursive = True

  # 爬取內文
  article = ''
  try:
    aas = soup.find(info_block_element, class_ = info_class)
    for a in aas.find_all(info_paragraph, recursive = recursive):
      # print(a)
      article += a.get_text()
    # allcontent.append(content)
  except:
    pass

  return article

In [15]:
to_run_list = creat_list_to_run_(input_folder, output_folder)  #載入待處理清單

for filename in to_run_list:
  df = pd.read_csv(rf'C:\Users\RYAN\OneDrive\桌面\Python\Capstone_玉山銀行v3\{work_folder}\{input_folder}\{filename}', encoding = 'UTF-8-Sig')
  df['text'] = None
  name = filename.split('_')[0]
  for i in range(len(df['title'])):
    try:
      info_url = df['urls'][i]
      ref = df['ref'][i]
      df.loc[i, 'text'] = grabing(info_url, ref, info)

    except:
      df.loc[i, 'text'] = None

    if i % 10 == 0:
      df.to_csv(rf'C:\Users\RYAN\OneDrive\桌面\Python\Capstone_玉山銀行v3\{work_folder}\{output_folder}\{name}_TEXT.csv', encoding = 'UTF-8-Sig', index=False)
      
  print(f"{name}爬取到內文的新聞篇數: {len(df['text']) - list(df.text).count(None)}")
  df.to_csv(rf'C:\Users\RYAN\OneDrive\桌面\Python\Capstone_玉山銀行v3\{work_folder}\{output_folder}\{name}_TEXT.csv', encoding = 'UTF-8-Sig', index=False)

央行爬取到內文的新聞篇數: 110
美元爬取到內文的新聞篇數: 137
美國PCE爬取到內文的新聞篇數: 21
美國十年期公債殖利率爬取到內文的新聞篇數: 32
美國非農就業爬取到內文的新聞篇數: 15
金管會爬取到內文的新聞篇數: 153


In [16]:
end_time = time.time()
total_time001 = end_time - start_time

# 將 total_time 轉換為秒數
total_seconds = int(total_time001)

# 計算小時數
hours = total_seconds // 3600

# 計算剩餘秒數
remaining_seconds = total_seconds % 3600

# 計算分鐘數
minutes = remaining_seconds // 60

# 計算剩餘秒數
seconds = remaining_seconds % 60

# 格式化輸出
output = f"{hours:02d}hrs{minutes:02d}mins{seconds:02d}secs"

print("程式執行時間：", output)

程式執行時間： 00hrs39mins47secs


# 002_第一階段摘要(ChatGPT串接)

In [17]:
# 設定工作環境
input_folder = '001_新聞內文'
output_folder = '002_第一階段摘要'
file_suffix_name = '1th_response(ChatGPT)'
start_time = time.time()
# 設定參數與提示詞
input_tokens_limit = 8192
parameter = {
        'temperature': 0,
        'top_p': 0.2,
        'max_tokens': 4096
        }

prompt1 = '''具備分析及研究價值的新聞文本。
        任務：產出新聞之摘要
        格式1：以數字型條列出重點小標題
        格式2：每一重點小標題內，以*進行分點敘述
        提示1：確保所有數字的準確性
        提示2：切勿產出任何文本未提及的內容
        提示3：需囊括新聞A之所有"重點"
        提示4：全部字數50字以內
        提示5：嚴格遵守下列格式

        格式：
        1. XXX
            * xxx
            * xxx
        2. XXX
            * xxx
            * xxx
        3. 以此類推...

        新聞：
        '''

prompt2 = '''財報具備分析及預測價值的新聞文本。請確保所有財務數字的準確性，且勿產出任何文本未提及的內容。
      **摘要要求**
      * 摘要的長度：100字以內。
      * 摘要的格式1：應該以數字型條列出重點小標題
      * 摘要的格式2：應於每一重點小標題內，以*進行分點敘述
      * 摘要的內容：應涵蓋所有新聞的主要內容，包括5W1H（何人、何事、何時、何地、為何、如何）。
      * 摘要的語氣：應客觀公正，避免加入個人意見或偏見。

      **其他注意事項**
      * 請盡可能使用簡潔明瞭的語言。
      * 請確保摘要的準確性和完整性。

      **示例**
      ## 新聞摘要
      1. XXX
        * xxx
        * xxx
      2.XXX
        * xxx
        * xxx
      3. 以此類推...
      (2)	請為新聞文本選擇最貼近其整體內容的一個「主題標籤」，「主題標籤」選項如下：
      A. 銀行業業務相關：該新聞通常涉及銀行業務、金融市場或金融法規相關資訊，可能涵蓋的主題包括但不限於：
      (a) 銀行業務的創新，如：新信用卡的推出
      (b) 新興金融科技的導入，如：數位轉型。
      (c) 銀行業務或組織的變革，如：合併與收購。
      (d) 金融市場的波動，如：股票、債券、外匯的波動。
      (e) 業內投資大幅調整，銀行投資部位大幅調整
      (f) 金融監管的裁罰及調整（不含永續相關者），如：年度風險管理要求
      (g) 銀行卡優惠及合作，如：信用卡與航空公司、第三方支付的合作
      (h) 跨業合作，如：信用卡與餐旅業、外送平台、零售業等等的合作
      B. 股市相關：該新聞講述的內容必須涵蓋股價表現，其通常涉及各方投資人對該公司之看好或看壞，抑或是買超或賣超。可能涵蓋的主題包括但不限於：
      (a) 銀行的股價波動，如：開盤價、收盤價
      (b) 分析師評級與投資建議
      (c) 市場前景與趨勢分析，如：專家分析
      C. 永續相關：該新聞通常涉及銀行在環境、社會和治理（ESG）方面的表現、倡議或活動等相關資訊。可能涵蓋的主題包括但不限於：
      (a) ESG指標、報告級績效相關議題
      (b) 永續金融商品或服務之推動，如：綠色貸款、社會企業之投資
      (c) 社會責任活動的舉行，如：舉辦慈善捐款、淨灘、體育等與銀行業務無關的活動
      (d) 永續發展策略和目標的訂定
      (e) 氣候相關風險管理
      D. 財務相關：該新聞通常涉及銀行財務狀況、財報公告、財務績效等相關資訊，其可協助投資者、分析師和其他利害相關人了解銀行的財務表現和市場地位。可能涵蓋的主題包括但不限於：
      (a) 財報之公布
      (b) 獲利情況及相關財務指標
      (c) 資本市場活動，如：新股及債券之發行、股票買回
      (d) 法說會內容截取或摘要

      新聞文本：
      '''

for_count_token_ = '''(
        model="gpt-3.5-turbo",
        messages=[
          {"role": "system", "content": "你是一個使用zhtw為主要語言的機器人助理，職責在於將新聞摘要成格式化的完整摘要"},
          {"role": "user", "content": "Hello!"}
        ],
        temperature=self.parameter['temperature'],
        top_p=self.parameter['top_p'],
        max_tokens= parameter['max_tokens']
      )'''

In [23]:
class ChatGPT_API_:

  def __init__(self, input_tokens_limit: int, base_prompt: str, parameter: dict):
    self.input_tokens_limit = input_tokens_limit
    self.base_prompt = base_prompt
    self.parameter = parameter
    self.client = self.start_()

  # 載入套件、存取API、設定model
  def start_(self):
    # 設置聊天
    client = OpenAI(api_key = "")
    return client

  # 進度條 # from GPT
  def progress_bar_(self, total_length, current_progress):
    bar_length = 30
    progress = current_progress / total_length
    block = int(round(bar_length * progress))
    text = "\r[{}] {:.2f}%".format("▮" * block + "▯" * (bar_length - block), progress * 100)
    sys.stdout.write(text)
    sys.stdout.flush()

  # 計算token
  # 避免Prompt總字數超過API的token限制，而餵不進LLM
  # from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
  def num_tokens_from_string_(self, string: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding('cl100k_base')   # 計算token的基礎model，須隨模型發展而調整
    num_tokens = len(encoding.encode(string))
    return num_tokens

  # 字串切割 # form GPT
  def split_string_(self, text):
    split_texts = []   # 準備存放切割後段落
    tokens_limit = self.input_tokens_limit  # 設定input tokens上限

    while self.num_tokens_from_string_(text+self.base_prompt+for_count_token_) > tokens_limit:
      split_index = text.rfind("。", 0, tokens_limit-self.num_tokens_from_string_(self.base_prompt+for_count_token_))
      if split_index == -1:
          split_index = tokens_limit
      split_texts.append(text[:split_index+1])
      text = text[split_index+1:]

    split_texts.append(text)

    return split_texts

  # 輸入問題，取得答覆
  def interact_(self, paragraph, name):
    try:
      # 互動
      completion = self.client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
          {"role": "system", "content": f"你是一個使用zhtw為主要語言的機器人助理，職責在於將{name}的新聞摘要成格式化的完整摘要"},
          {"role": "user", "content": f"請根據以下多篇的{name+self.base_prompt+paragraph}"}
        ],
        temperature=self.parameter['temperature'],
        top_p=self.parameter['top_p'],
        max_tokens= parameter['max_tokens']
      )
      response = completion.choices[0].message.content
    except:
      response = '-'  # 可能會不定崩潰，崩潰時先以None填入，稍後再試

    return response

  # 遍歷所有新聞，並摘要之
  def abstracting_(self, df, name):
    # 初始化進度條
    current_progress = 0   # 進度條分子
    none_count = sum('-' in response for response in df['response'])   # 進度條分母

    for i, text in enumerate(df[df['response']=='-']['text']):   # 遍歷所有response的index和值
    # 避免新聞文本過長
      splited_doc = self.split_string_(text)    
      # 太長的再分開摘要，分開的摘要合併後再摘要一次
      if len(splited_doc)>1:
        i_res = ''
        for sp_i in splited_doc:
          i_res += self.interact_(sp_i, name)
        df.loc[i,'response']  = self.interact_(i_res, name)
      # 沒太長就直接摘
      else:
        df.loc[i,'response'] = self.interact_(splited_doc[0], name)
      time.sleep(2)   # 3.5-turbo請求上限：60,000TPM or 500RPM (一篇新聞大概2000Tokens 差不多每分鐘30篇便接近上限)
      current_progress+=1
      self.progress_bar_(none_count, current_progress)

      # 避免執行環境之意外，每10筆回應便暫存一次
      if current_progress % 10 == 0:
        df.to_csv(rf'C:\Users\RYAN\OneDrive\桌面\Python\Capstone_玉山銀行v3\{work_folder}\{output_folder}\{name}_{file_suffix_name}.csv', encoding = 'UTF-8-Sig', index=False)
    return df

  # 合併摘要（與002程式之差異）
  def conbine_abstract_(self, df):
    conbine = ''
    conbine_list = []
    for i in range(len(df['response'])):
      # 避免合併摘要過長
      if self.num_tokens_from_string_(self.base_prompt + conbine + df['response'][i]) < self.input_tokens_limit:
        conbine += df['response'][i]
        if i == len(df['response'])-1:  # 處理未達token上限便合併完者
          conbine_list.append(conbine)
      else:
        conbine_list.append(conbine)
        conbine = ''
    return conbine_list

  # 摘要收斂（與002程式之差異）
  def abstract_convergencing_(self, df, name):
    print(f'\n{name} is processing')
    Originally_it_was_only_one=False
    # 有些tag可能只有一篇新聞，但還是得進迴圈
    if len(df['response']) == 1:
      Originally_it_was_only_one = True
    while len(df['response']) > 1 or Originally_it_was_only_one:
      # 產出合併摘要之list
      temp_list = self.conbine_abstract_(df)
      # 創建一個新的df存放合併摘要 # 初始化text及response
      df = pd.DataFrame({'text':temp_list,
                         'response':'-'
      })
      
      # 針對合併摘要進行摘要
      df = self.abstracting_(df, name)
      # for docx
      output_doc = df['response'].apply(lambda x: ''.join(map(str, x)))  # 將list轉換成字串(無'['跟']')，以照格式寫進docx
      # for while loop
      df['response'] = df['response'].astype(str)   # 將list轉換成str回存，以符合conbine_abstract_(df)裡，self.num_tokens_from_string_(df['response'][i])的格式要求。
      Originally_it_was_only_one = False
      
    return output_doc, df

In [19]:
to_run_list = creat_list_to_run_(input_folder, output_folder)  #載入待處理清單

# ChatGPT.keep_doing()   # 先回頭處理先前中斷之檔案

# 逐檔執行主程式
for filename in to_run_list:
  name = filename.split('_')[0]
  print(f'{name} is processing')

  if (name[-2:] == '銀行') or (name[-2:] == '世華'):
    base_prompt = prompt2
    print('use the prompt2')
  else:
    base_prompt = prompt1
    print('use the prompt1')

  # 呼叫 LLM
  ChatGPT = ChatGPT_API_(input_tokens_limit, base_prompt, parameter)  # 初始化LLM
  ChatGPT.start_()

  df = pd.read_csv(rf"C:\Users\RYAN\OneDrive\桌面\Python\Capstone_玉山銀行v3\{work_folder}\{input_folder}\{filename}")
  # 篩出資料型態為 str 的資料
  df = df[df['text'].apply(lambda x: isinstance(x, str))]
  df.reset_index(drop=True, inplace=True)  # 重設index（篩出後資料的index是對應舊資料）

  # 針對完全未跑過的檔案初始化response欄位，填上df長度的none，使主程式能對所有df資料進行處理
  df['response'] = '-'
  df['tokens'] = df['text'].apply(lambda x: ChatGPT.num_tokens_from_string_(x+base_prompt+for_count_token_) )

  df = ChatGPT.abstracting_(df, name)

  if name[-2:] == '銀行' or name[-2:] == '世華':
    df['tag'] = 0
    for index, response in enumerate(df['response']):
      try:
        df['tag'][index] = re.findall(r"銀行業業務相關|股市相關|永續相關|財務相關", str(response))[0]
      except:
        df['tag'][index] = '其他'

  # 另存新檔
  df.to_csv(rf'C:\Users\RYAN\OneDrive\桌面\Python\Capstone_玉山銀行v3\{work_folder}\{output_folder}\{name}_{file_suffix_name}.csv', encoding = 'UTF-8-Sig', index=False)
  print(f'{name} was finished!!' )

FED is processing
use the prompt1
[▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮] 100.00%FED was finished!!
ICS2.0 is processing
use the prompt1
[▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮] 100.00%ICS2.0 was finished!!
IFRS17 is processing
use the prompt1
IFRS17 was finished!!
利率決策 is processing
use the prompt1
[▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮] 100.00%利率決策 was finished!!
台灣CPI is processing
use the prompt1
[▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮] 100.00%台灣CPI was finished!!
外匯 is processing
use the prompt1
[▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮] 100.00%外匯 was finished!!
央行 is processing
use the prompt1
[▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮] 100.00%央行 was finished!!
美元 is processing
use the prompt1
[▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮] 100.00%美元 was finished!!
美國PCE is processing
use the prompt1
[▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮] 100.00%美國PCE was finished!!
美國十年期公債殖利率 is processing
use the prompt1
[▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮] 100.00%美國十年期公債殖利率 was finished!!
美國非農就業 is processing
use the prompt1
[▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮] 100.00%美國非

In [20]:
end_time = time.time()
total_time002 = end_time - start_time

# 將 total_time 轉換為秒數
total_seconds = int(total_time002)

# 計算小時數
hours = total_seconds // 3600

# 計算剩餘秒數
remaining_seconds = total_seconds % 3600

# 計算分鐘數
minutes = remaining_seconds // 60

# 計算剩餘秒數
seconds = remaining_seconds % 60

# 格式化輸出
output = f"{hours:02d}hrs{minutes:02d}mins{seconds:02d}secs"

print("程式執行時間：", output)

程式執行時間： 01hrs07mins15secs


# 003_第二階段摘要(ChatGPT串接)

In [24]:
# 設定工作環境
input_folder = '002_第一階段摘要'
output_folder = '003_第二階段摘要'
file_suffix_name = '2th_response(ChatGPT)'
start_time = time.time()
# 設定參數與提示詞
input_tokens_limit = 8192
parameter = {
        'temperature': 0,
        'top_p': 0.2,
        'max_tokens': 4096
        }

base_prompt = '''新聞摘要，以union方式結合，以撰寫一篇A4大小的條列式文字摘要。摘要應涵蓋所有新聞的主要內容。摘要的語氣應客觀公正，避免加入個人意見或偏見。
        **摘要要求**
        * 摘要的長度：應為A4大小（約1頁）。
        * 摘要的格式1：應該以數字型條列出重點小標題
        * 摘要的格式2：應於每一重點小標題內，以*進行分點敘述
        * 摘要的內容：應涵蓋所有新聞的主要內容，包括5W1H（何人、何事、何時、何地、為何、如何）。
        * 摘要的語氣：應客觀公正，避免加入個人意見或偏見。

        **其他注意事項**
        * 請盡可能使用簡潔明瞭的語言。
        * 請確保摘要的準確性和完整性。
        * 請以摘要主角為重心。

        **示例**
        ## 新聞摘要
        1.XXX
          * xxx
          * xxx
        2.XXX
          * xxx
          * xxx
        3. 以此類推...

        請根據以上要求，撰寫一篇A4大小的文字摘要。
        **各篇新聞摘要如下**
        '''

for_count_token_ = '''(
        model="gpt-3.5-turbo",
        messages=[
          {"role": "system", "content": f"你是一個使用zhtw為主要語言的機器人助理，職責在於將{name}的新聞摘要成格式化的完整摘要"},
          {"role": "user", "content": f"請根據以下多篇的{name+self.base_prompt+paragraph}"}
        ],
        temperature=self.parameter['temperature'],
        top_p=self.parameter['top_p'],
        max_tokens= parameter['max_tokens']
      )'''


In [27]:
# 載入關鍵字新聞檔案
current_progress = 0  # 初始化進度條
to_run_list = creat_list_to_run_(input_folder, output_folder)  #載入待處理清單
ChatGPT = ChatGPT_API_(input_tokens_limit, base_prompt, parameter)  # 初始化LLM
ChatGPT.start_()

# 主程式
# run所有關鍵字檔案，將摘要合併並產出最終摘要
for filename in to_run_list:
  df_0 = pd.read_csv(rf"C:\Users\RYAN\OneDrive\桌面\Python\Capstone_玉山銀行v3\{work_folder}\{input_folder}\{filename}")
  name_0 = filename.split('_')[0]
  if len(df_0)<1:
    print(f'{name_0} have no text')
    continue
  # 針對銀行df進行tag分類，讓每一個tag都有一份二階摘要
  if name_0[-2:] == '銀行' or name_0[-2:] == '世華':
    df = pd.DataFrame()
    for i in df_0['tag'].unique():
      name = name_0 + '_(' + i + ')'  # for DOC檔案命名 e.g.國泰世華_(財務相關)
      df_tag = df_0[df_0['tag']==i]
      df_tag = df_tag.reset_index()
      output_doc, temp_df = ChatGPT.abstract_convergencing_(df_tag, name)
      temp_df['tag'] = i
      df = pd.concat([df, temp_df], axis=0)
      # 產出最終摘要（DOC檔）（004用此檔）
      with open(rf"C:\Users\RYAN\OneDrive\桌面\Python\Capstone_玉山銀行v3\{work_folder}\{output_folder}\{name}_{file_suffix_name}.docx", "w") as file:
        for i in output_doc:
          file.write(i)
    print('cluster of tag')
  else:
    output_doc, df = ChatGPT.abstract_convergencing_(df_0, name_0)
    # 產出最終摘要（DOC檔）（004用此檔）
    with open(rf"C:\Users\RYAN\OneDrive\桌面\Python\Capstone_玉山銀行v3\{work_folder}\{output_folder}\{name_0}_{file_suffix_name}.docx", "w") as file:
      for i in output_doc:
        file.write(i)

  # 產出最終摘要（CSV）
  df.to_csv(rf"C:\Users\RYAN\OneDrive\桌面\Python\Capstone_玉山銀行v3\{work_folder}\{output_folder}\CSV\{name_0}_{file_suffix_name}.csv", encoding = 'UTF-8-Sig', index=False)

  print(f'\n !!{name_0} was finished!!' )


利率決策 is processing
[▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮] 100.00%
 !!利率決策 was finished!!

台灣CPI is processing
[▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮] 100.00%
 !!台灣CPI was finished!!

外匯 is processing
[▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮] 100.00%
 !!外匯 was finished!!

央行 is processing
[▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮] 100.00%
 !!央行 was finished!!

美元 is processing
[▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮] 100.00%
 !!美元 was finished!!

美國PCE is processing
[▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮] 100.00%
 !!美國PCE was finished!!

美國十年期公債殖利率 is processing
[▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮] 100.00%
 !!美國十年期公債殖利率 was finished!!

美國非農就業 is processing
[▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮] 100.00%
 !!美國非農就業 was finished!!

金管會 is processing
[▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮] 100.00%
 !!金管會 was finished!!


In [28]:
end_time = time.time()
total_time003 = end_time - start_time

# 將 total_time 轉換為秒數
total_seconds = int(total_time003)

# 計算小時數
hours = total_seconds // 3600

# 計算剩餘秒數
remaining_seconds = total_seconds % 3600

# 計算分鐘數
minutes = remaining_seconds // 60

# 計算剩餘秒數
seconds = remaining_seconds % 60

# 格式化輸出
output = f"{hours:02d}hrs{minutes:02d}mins{seconds:02d}secs"

print("程式執行時間：", output)

程式執行時間： 00hrs22mins55secs


# 004_摘要回溯

In [29]:
file_suffix_name2 = '2th_response(ChatGPT).docx'
# 設定工作環境
input_folder = '003_第二階段摘要'
output_folder = '004_摘要回溯'
file_suffix_name = "Final_Report"
start_time = time.time()

In [30]:
# 寫入excel
from openpyxl import Workbook

## 爬蟲
使用000_爬取新聞URL的函式

## Excel

In [31]:
# 寫入excel時，有些字會是excel限制的Illegal_Character，故遇上時需再加一個跳脫字元
# 程式碼參考自https://www.havnemark.dk/
def escape_xlsx_char(ch):
	illegal_xlsx_chars = {
	'\x00':'\\x00',	#	NULL
	'\x01':'\\x01',	#	SOH
	'\x02':'\\x02',	#	STX
	'\x03':'\\x03',	#	ETX
	'\x04':'\\x04',	#	EOT
	'\x05':'\\x05',	#	ENQ
	'\x06':'\\x06',	#	ACK
	'\x07':'\\x07',	#	BELL
	'\x08':'\\x08',	#	BS
	'\x0b':'\\x0b',	#	VT
	'\x0c':'\\x0c',	#	FF
	'\x0e':'\\x0e',	#	SO
	'\x0f':'\\x0f',	#	SI
	'\x10':'\\x10',	#	DLE
	'\x11':'\\x11',	#	DC1
	'\x12':'\\x12',	#	DC2
	'\x13':'\\x13',	#	DC3
	'\x14':'\\x14',	#	DC4
	'\x15':'\\x15',	#	NAK
	'\x16':'\\x16',	#	SYN
	'\x17':'\\x17',	#	ETB
	'\x18':'\\x18',	#	CAN
	'\x19':'\\x19',	#	EM
	'\x1a':'\\x1a',	#	SUB
	'\x1b':'\\x1b',	#	ESC
	'\x1c':'\\x1c',	#	FS
	'\x1d':'\\x1d',	#	GS
	'\x1e':'\\x1e',	#	RS
	'\x1f':'\\x1f'}	#	US

	if ch in illegal_xlsx_chars:
		return illegal_xlsx_chars[ch]

	return ch

# Wraps the function escape_xlsx_char(ch).
def escape_xlsx_string(st):

	return ''.join([escape_xlsx_char(ch) for ch in st])

In [32]:
'''報告寫進ws1指定儲存格'''
def write_ws():
  # 將每一句新聞摘要寫入到工作表的儲存格中
  for i, sentence in enumerate(each_sentences, start=1):  # 讓index從1開始而非0（配合excel的row index）
    ws1.cell(row=i+1, column=1, value=sentence)        # 根據index將each_sentences的每一句話依序寫入儲存格

    true_sentence = sentence.strip(' *。')       # 將空格、米字號、句號刪掉
    print(true_sentence)   # 大概檢查抓的結果
    if true_sentence in small_sentences:
      write_the_title(i, true_sentence, 1)

# 將新聞原文寫在摘要旁邊
def write_the_title(i, true_sentence, n):
    # 動態欄位index
    start_column = (n - 1) * 4 + 2
    end_column = start_column + 3

    # 爬蟲
    true_sentence_cleaned = re.sub(r'[^\w\s.%]', '', true_sentence)
    if n == 1:
      true_sentence_cleaned = true_sentence_cleaned
    elif n == 2:
      if '，' in true_sentence:
        true_sentence_cleaned = true_sentence.split('，')[0]
      else:
        true_sentence_cleaned = true_sentence_cleaned[:len(true_sentence_cleaned)//2+1]
    else:
      if '，' in true_sentence:
        true_sentence_cleaned = true_sentence.split('，')[1]
      else:
        true_sentence_cleaned = true_sentence_cleaned[len(true_sentence_cleaned)//2+1:]
    
    title, date, url, ref, no_use_page_number = get_urls(true_sentence_cleaned, 8)
    if title==[]:
      title=['-']
    if date==[]:
      date=['-']
    if url==[]:
      url=['-']
    if ref==[]:
      ref=['-']
  
    ws1.cell(row=i+1, column=start_column, value=title[0])
    ws1.cell(row=i+1, column=start_column+1, value=date[0])
    # url以超連結型態寫入
    if url[0].startswith("http"):
      ws1.cell(row=i+1, column=start_column+2).hyperlink = url[0]
    else:
      ws1.cell(row=i+1, column=start_column+2, value=url[0])
    ws1.cell(row=i+1, column=end_column, value=ref[0])


'''ws1寫進exxcel'''
def write_to_excel(final_abstract):
  global each_sentences
  global small_sentences
  each_sentences = re.findall(r"(.*?)[\n。$]", final_abstract, re.MULTILINE)         # 抓出最終摘要的每一句話（連大標小標都抓）
  small_temp_sentences = re.findall(r"^\s+\*\s+(?:.*?)[\n。$]", final_abstract, re.MULTILINE)    # 抓出最終摘要的每一句摘要（不抓大標小標）
  # 整理 small_sentences
  small_sentences = []
  for sentence in small_temp_sentences:
    cleaned_sentence = sentence.strip(' *。\n')
    small_sentences.append(cleaned_sentence)

  # print(each_sentences)   # 大概檢查抓的結果
  # print(small_sentences)

  # 創建一個新的excel
  global wb, ws1
  wb = Workbook()
  # 創建sheet
  ws1 = wb.active
  ws1.title = "Final_Abstract"   # 更改預設sheet的名稱

  # 設定欄位名稱
  ws1.cell(row=1, column=1).value = '新聞摘要'
  for n in range(3):
    start_column = n * 4 + 2
    n = str(n+1)
    values = ['原文標題'+n, '原文時間'+n, '原文網址'+n, '原文出處'+n]
    for i, value in enumerate(values):
      ws1.cell(row=1, column=start_column+i).value = value

  write_ws()
  for i, sentence in enumerate(each_sentences, start=1):
    if ws1.cell(row=i+1, column=2).value == '-':
      write_the_title(i ,ws1.cell(row=i+1, column=1).value.strip(' *。'), 2)
  for i, sentence in enumerate(each_sentences, start=1):
    if ws1.cell(row=i+1, column=6).value == '-':
      write_the_title(i, ws1.cell(row=i+1, column=1).value.strip(' *。'), 3)

  # 匯出
  wb.save(rf"C:\Users\RYAN\OneDrive\桌面\Python\Capstone_玉山銀行v3\{work_folder}\{output_folder}\{name}_{file_suffix_name}.xlsx") 

## Run

In [33]:
to_run_list = creat_list_to_run_(input_folder, output_folder)  #載入待處理清單
to_run_list.remove('CSV')
for filename in to_run_list:
  # 讀取最終摘要文字檔
  name = filename.split('_')[0]
  if name[-2:] == '銀行' or name[-2:] == '世華':
    name = filename.split('_')[0]+'_'+filename.split('_')[1]
  print(f'{name} is processing')
  with open(rf"C:\Users\RYAN\OneDrive\桌面\Python\Capstone_玉山銀行v3\{work_folder}\{input_folder}\{name}_{file_suffix_name2}", "r") as file:
    final_abstract = file.read()
    write_to_excel(final_abstract)

FED is processing
## 新聞摘要

1. **黃金價格波動
現貨黃金漲 1.1%，至每盎司 2353.90 美元
https://www.google.com/search?q=現貨黃金漲 1.1%至每盎司 2353.90 美元&start=0&tbs=sbd:1,qdr:m,sbd:1&tbm=nws
False 30
2024-06-06 06:58:32.315475
8 月交割的黃金期貨漲 1.1%，至每盎司 2374.10 美元
https://www.google.com/search?q=8 月交割的黃金期貨漲 1.1%至每盎司 2374.10 美元&start=0&tbs=sbd:1,qdr:m,sbd:1&tbm=nws
False 30
2024-06-06 06:58:41.409468
黃金價格在5月20日觸及2449.89美元的歷史新高
https://www.google.com/search?q=黃金價格在5月20日觸及2449.89美元的歷史新高&start=0&tbs=sbd:1,qdr:m,sbd:1&tbm=nws
False 30
2024-06-04 15:58:51.407599
2024-06-04 15:58:51.409600
2024-05-23 15:58:51.419601

2. **美國經濟及Fed政策
美國私人就業新增人數低於預期，公債殖利率下跌至4月以來最低水準
https://www.google.com/search?q=美國私人就業新增人數低於預期公債殖利率下跌至4月以來最低水準&start=0&tbs=sbd:1,qdr:m,sbd:1&tbm=nws
False 30
2024-06-06 08:59:00.175399
2024-06-06 06:59:00.183385
多數經濟學家預測Fed將在9月開始降息，但一些人認為可能只降息一次或根本不降息
https://www.google.com/search?q=多數經濟學家預測Fed將在9月開始降息但一些人認為可能只降息一次或根本不降息&start=0&tbs=sbd:1,qdr:m,sbd:1&tbm=nws
False 30
2024-06-06 02:59:05.630790
Fed可能在年底前降息，提高黃金投資吸

In [34]:
end_time = time.time()
total_time004 = end_time - start_time

# 將 total_time 轉換為秒數
total_seconds = int(total_time004)

# 計算小時數
hours = total_seconds // 3600

# 計算剩餘秒數
remaining_seconds = total_seconds % 3600

# 計算分鐘數
minutes = remaining_seconds // 60

# 計算剩餘秒數
seconds = remaining_seconds % 60

# 格式化輸出
output = f"{hours:02d}hrs{minutes:02d}mins{seconds:02d}secs"

print("程式執行時間：", output)

程式執行時間： 01hrs06mins06secs


# 005_合併摘要

In [35]:
# 設定工作環境
input_folder = '004_摘要回溯'
output_folder = '005_各檔案合併_forTableau'
file_suffix_name = "forTableau"
start_time = time.time()

In [36]:
file_list = os.listdir(rf"C:\Users\RYAN\OneDrive\桌面\Python\Capstone_玉山銀行v3\{work_folder}\{input_folder}")
all_bank_df = pd.DataFrame(columns=['新聞摘要', '原文標題1', '原文時間1', '原文網址1', '原文出處1', '原文標題2', '原文時間2', '原文網址2', '原文出處2', '原文標題3', '原文時間3', '原文網址3', '原文出處3', '關鍵字'])
all_macroeconomics_df = pd.DataFrame(columns=['新聞摘要', '原文標題1', '原文時間1', '原文網址1', '原文出處1', '原文標題2', '原文時間2', '原文網址2', '原文出處2', '原文標題3', '原文時間3', '原文網址3', '原文出處3', '關鍵字'])
for filename in file_list:
  if filename[-4:] == 'xlsx':
    name = filename.split('_')[0]
    df = pd.read_excel(rf"C:\Users\RYAN\OneDrive\桌面\Python\Capstone_玉山銀行v3\{work_folder}\{input_folder}\{filename}", sheet_name = 'Final_Abstract')
    df['關鍵字'] = name
    df['指定摘要時段'] = work_folder[-6:]
    # 銀行
    if name[-2:] == '銀行' or name[-2:] == '世華':
      tag_name = name + filename.split('_')[1]
      df['關鍵字'] = tag_name
      all_bank_df = pd.concat([all_bank_df, df], axis=0)
      print(f'{tag_name} is concated to the 【bank】')
    else:
      all_macroeconomics_df = pd.concat([all_macroeconomics_df, df], axis=0)
      print(f'{name} is concated to the 【macroeconomics】')

all_bank_df.to_excel(rf"C:\Users\RYAN\OneDrive\桌面\Python\Capstone_玉山銀行v3\{work_folder}\{output_folder}\all_bank_{file_suffix_name}.xlsx", index_label='index')
all_macroeconomics_df.to_excel(rf"C:\Users\RYAN\OneDrive\桌面\Python\Capstone_玉山銀行v3\{work_folder}\{output_folder}\all_macroeconomics_{file_suffix_name}.xlsx", index_label='index')

FED is concated to the 【macroeconomics】
ICS2.0 is concated to the 【macroeconomics】
中信銀行(永續相關) is concated to the 【bank】
中信銀行(股市相關) is concated to the 【bank】
中信銀行(財務相關) is concated to the 【bank】
中信銀行(銀行業業務相關) is concated to the 【bank】
兆豐銀行(永續相關) is concated to the 【bank】
兆豐銀行(股市相關) is concated to the 【bank】
兆豐銀行(財務相關) is concated to the 【bank】
兆豐銀行(銀行業業務相關) is concated to the 【bank】
利率決策 is concated to the 【macroeconomics】
台北富邦銀行(永續相關) is concated to the 【bank】
台北富邦銀行(股市相關) is concated to the 【bank】
台北富邦銀行(財務相關) is concated to the 【bank】
台北富邦銀行(銀行業業務相關) is concated to the 【bank】
台新銀行(永續相關) is concated to the 【bank】
台新銀行(股市相關) is concated to the 【bank】
台新銀行(財務相關) is concated to the 【bank】
台新銀行(銀行業業務相關) is concated to the 【bank】
台灣CPI is concated to the 【macroeconomics】
國泰世華(股市相關) is concated to the 【bank】
國泰世華(財務相關) is concated to the 【bank】
國泰世華(銀行業業務相關) is concated to the 【bank】
外匯 is concated to the 【macroeconomics】
央行 is concated to the 【macroeconomics】
玉山銀行(永續相關) is concated to the 

<ipython-input-36-d21a2df98f75>:20: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.2.9' currently installed).
  all_bank_df.to_excel(rf"C:\Users\RYAN\OneDrive\桌面\Python\Capstone_玉山銀行v3\{work_folder}\{output_folder}\all_bank_{file_suffix_name}.xlsx", index_label='index')
<ipython-input-36-d21a2df98f75>:21: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.2.9' currently installed).
  all_macroeconomics_df.to_excel(rf"C:\Users\RYAN\OneDrive\桌面\Python\Capstone_玉山銀行v3\{work_folder}\{output_folder}\all_macroeconomics_{file_suffix_name}.xlsx", index_label='index')


# 執行時間

In [37]:
end_time = time.time()
total_time = end_time - start_time

# 將 total_time 轉換為秒數
total_seconds = int(total_time)

# 計算小時數
hours = total_seconds // 3600

# 計算剩餘秒數
remaining_seconds = total_seconds % 3600

# 計算分鐘數
minutes = remaining_seconds // 60

# 計算剩餘秒數
seconds = remaining_seconds % 60

# 格式化輸出
output = f"{hours:02d}hrs{minutes:02d}mins{seconds:02d}secs"

print("程式執行時間：", output)

程式執行時間： 00hrs00mins04secs


202402銀行：6家，耗時 3 hr 18 min

202401銀行：6家，耗時 3 hr 37 min